In [ ]:
!pip install transformers datasets peft bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import torch as torch
from transformers import AutoTokenizer, AutoModelForCausalLM,Gemma3ForCausalLM,BitsAndBytesConfig
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
from datasets import load_dataset
#from torchvision import datasets

In [4]:
data=load_dataset("OpenAssistant/oasst1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-b42a775f407cee45.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

(…)-00000-of-00001-134b8fd0c89408b6.parquet:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [ ]:
data['train'][0]

In [5]:

from collections import defaultdict
# Group by message_id manually
conversations = defaultdict(list)
conversations_v = defaultdict(list)
for row in data['train']:
    conversations[row["message_tree_id"]].append((row["role"], row["text"]))

# Convert to list of dicts for Dataset
formatted = []
for convo_id, turns in conversations.items():
    convo_parts = []
    for role, content in turns:
        if role == "prompter":
            convo_parts.append(f"<start_of_turn>\n{content.strip()}<end_of_turn>")
        elif role == "assistant":
            convo_parts.append(f"<start_of_turn>\n{content.strip()}<end_of_turn>")
    chat_str = "<bos>" + "\n".join(convo_parts) + "<eos>"
    formatted.append({"input_ids":chat_str})

for row in data['validation']:
    conversations_v[row["message_tree_id"]].append((row["role"], row["text"]))

# Convert to list of dicts for Dataset
formatted_v = []
for convo_id, turns in conversations_v.items():
    convo_parts = []
    for role, content in turns:
        if role == "prompter":
            convo_parts.append(f"<start_of_turn>\n{content.strip()}<end_of_turn>")
        elif role == "assistant":
            convo_parts.append(f"<start_of_turn>\n{content.strip()}<end_of_turn>")
    chat_str = "<bos>" + "\n".join(convo_parts) + "<eos>"
    formatted_v.append({"input_ids":chat_str})


In [6]:
from datasets import Dataset, load_dataset
train=Dataset.from_list(formatted)
validate=Dataset.from_list(formatted_v)

In [7]:
DataLoaders=DataLoader(data,batch_size=8) # Not required with hugging face Trainer

In [7]:
from huggingface_hub import login
login(token = '')

In [8]:
# Not required as of now
from torch.utils.data import Dataset
class dataset(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data=data
        self.tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
    def __len__(self):
        return len(self.data['train'])
    def __getitem__(self,idx):
        prompt = f"### Instruction:\n{self.data['train'][idx]['instruction']}\n### Input:\n{self.data['train'][idx]['input']}\n### Response:\n{self.data['train'][idx]['output']}"
        tokenized_prompt = self.tokenizer(prompt, return_tensors="pt")
        return tokenized_prompt


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [9]:

model_id = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_bos_token=False
tokenizer.add_eos_token=False
#special_tokens = ["|assistant|", "|prompter|"]
#tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [10]:
def tokenize_fn(examples):
    return tokenizer(
        examples["input_ids"],
        truncation=True,
        max_length=256,
        padding=False  # Let data_collator handle padding
    )

train = train.map(tokenize_fn, batched=True)
validate = validate.map(tokenize_fn, batched=True)

Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [11]:


# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Prepare model for LoRA
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)



config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [13]:
# Count all parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"Total parameters: {total_params}")

# Count only trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Trainable parameters: {trainable_params}")

print(f"Non-trainable parameters: {trainable_params * 100 / total_params}")

Total parameters: 1517111296
Trainable parameters: 1843200
Non-trainable parameters: 0.1214940528661122


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./content/sample_data/logs/peft",       # Where to save checkpoints
    per_device_train_batch_size=4,     # Adjust for GPU memory\
    gradient_accumulation_steps=8,     # Increase effective batch size
    learning_rate=2e-4,                # PEFT can use higher LR
    weight_decay=0.01,                  # Regularization
    num_train_epochs=1,                 # Adjust for dataset size
   # max_length=512,                     # Max sequence length
    warmup_ratio=0.03,                  # Warmup fraction
    lr_scheduler_type="cosine",         # Common choice for LLM tuning
    logging_dir="./content/sample_data/logs/log",
    logging_steps=10,                   # Log every 10 steps
    save_strategy="steps",              # Save model periodically
    save_steps=200,
   # eval="steps",        # Evaluate periodically
    eval_steps=200,
    save_total_limit=2,                 # Keep only last 2 checkpoints
    #fp16=True,                          # Mixed precision
    #bf16=False,                         # If GPU supports bf16, set True
    gradient_checkpointing=True,        # Saves memory at cost of compute
    optim="adamw_torch",                # AdamW optimizer
    report_to=None,            # Or "wandb" for Weights & Biases
    remove_unused_columns=False,        # Needed for causal LM
    push_to_hub=False,
    # Upload to HF Hub if needed
)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=validate,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipython-input-1292345263.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: punit-sobti (punit-sobti-check) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.788000
20,3.009500
30,2.632100
40,2.462000
50,2.309600
60,2.313100
70,2.262000
80,2.299800
90,2.195500
100,2.178600


Step,Training Loss
10,3.788000
20,3.009500
30,2.632100
40,2.462000
50,2.309600
60,2.313100
70,2.262000
80,2.299800
90,2.195500
100,2.178600


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MoE(nn.Module):
    def __init__(self, d_model, d_ffn, num_experts, k=1):
        super().__init__()
        self.num_experts = num_experts
        self.k = k

        # Experts: list of independent FFNs
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(d_model, d_ffn),
                nn.ReLU(),
                nn.Linear(d_ffn, d_model)
            ) for _ in range(num_experts)
        ])

        # Gating network
        self.gate = nn.Linear(d_model, num_experts)

    def forward(self, x):
        # x: [batch, seq, d_model]
        b, l, d = x.shape

        # Gating scores
        gate_logits = self.gate(x)  # [b, l, num_experts]
        gate_probs = F.softmax(gate_logits, dim=-1)  # per-token distribution

        # Pick top-k experts per token
        topk_vals, topk_idx = torch.topk(gate_probs, self.k, dim=-1)  # [b, l, k]

        print("top_vals",topk_vals.shape)
        print("top_k",topk_idx)

        # Output buffer
        output = torch.zeros_like(x)

        for expert_id in range(self.num_experts):
            # Mask: which tokens route to this expert
            mask = (topk_idx == expert_id)  # [b, l, k]
            if not mask.any():
                continue
            print("mask",mask)
            # Get tokens for this expert
            mask_any = mask.any(dim=-1)  # [b, l]
            print("mask_any",mask_any)
            tokens = x[mask_any]  # [num_tokens_for_expert, d_model]
            print("tokens",tokens)

            # Apply expert
            out_tokens = self.experts[expert_id](tokens)
            print("out_tokens",out_tokens)

            # Scatter back
            output[mask_any] = out_tokens

        return output


In [ ]:
MoE(2,4,5)(torch.rand((3, 3, 2)))

top_vals torch.Size([3, 3, 1])
top_k tensor([[[3],
         [1],
         [2]],

        [[3],
         [3],
         [2]],

        [[3],
         [2],
         [3]]])
mask tensor([[[False],
         [ True],
         [False]],

        [[False],
         [False],
         [False]],

        [[False],
         [False],
         [False]]])
mask_any tensor([[False,  True, False],
        [False, False, False],
        [False, False, False]])
tokens tensor([[0.8047, 0.9771]])
out_tokens tensor([[-0.5167, -0.3423]], grad_fn=<AddmmBackward0>)
mask tensor([[[False],
         [False],
         [ True]],

        [[False],
         [False],
         [ True]],

        [[False],
         [ True],
         [False]]])
mask_any tensor([[False, False,  True],
        [False, False,  True],
        [False,  True, False]])
tokens tensor([[0.0841, 0.2653],
        [0.1111, 0.8782],
        [0.0574, 0.7352]])
out_tokens tensor([[ 0.1461, -0.0084],
        [ 0.1688,  0.0231],
        [ 0.1606,  0.0144]

tensor([[[ 0.3055, -0.0531],
         [-0.5167, -0.3423],
         [ 0.1461, -0.0084]],

        [[ 0.4441,  0.0438],
         [ 0.2889, -0.0647],
         [ 0.1688,  0.0231]],

        [[ 0.3452, -0.0253],
         [ 0.1606,  0.0144],
         [ 0.3761, -0.0037]]], grad_fn=<IndexPutBackward0>)